In [1]:
# Setting up google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')

Mounted at /content/gdrive


In [2]:
# import other dependencies
import torch
from torch import nn
# import my_utils as mu

import urllib.request
from os.path import isfile, isdir
import tarfile

# Task 1 - Read dataset and create dataloaders

In [3]:
# Get Dataset


# Define the CIFAR-10 dataset URL and folder path
cifar_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
cifar10_dataset_folder_path = 'cifar-10-batches-py'

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# check if the data file is already downloaded
# if not, download it from "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
# and save as cifar-10-python.tar.gz
if not isfile('cifar-10.tar.gz'):
  # Download the dataset
  print("Downloading CIFAR-10 dataset...")
  urllib.request.urlretrieve(cifar_url, "cifar-10.tar.gz")
  print("Download complete!")
else:
  print("Already downloaded")

# Extract the downloaded archive
if not isdir(cifar10_dataset_folder_path):
  print("Extracting CIFAR-10...")
  with tarfile.open("cifar-10.tar.gz", "r:gz") as tar_file:
    tar_file.extractall()
  print("Extraction complete!")
else:
  print("Already extracted")


Download complete!
Extracting CIFAR-10...
Extraction complete!


In [4]:
# Create Data loaders

from torchvision import datasets, transforms

# Define data directory
data_dir = "./"

# Define transformations
transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load train and test datasets
train_dataset = datasets.CIFAR10(root=data_dir, train=True, download=False, transform=transform)
test_dataset = datasets.CIFAR10(root=data_dir, train=False, download=False, transform=transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Task 2 - Create the Model

In [16]:
# Create the model


class Block(nn.Module):
  def __init__(self, num_inputs, k):
    super(Block, self).__init__()
    self.k = k
    self.spatialAveragePool = nn.AdaptiveAvgPool2d(output_size=(k, k))
    self.W = nn.Linear(k, num_inputs)
    self.conv_layers = nn.ModuleList([nn.Conv2d(3, 3, 1, 1) for _ in range(k)])

  def forward(self, x):
    a = nn.functional.relu(self.W(self.spatialAveragePool(x)))
    o = torch.sum(torch.stack([conv(x) for conv in self.conv_layers]), dim=0)
    return o

class Backbone(nn.Module):
  def __init__(self):
    super(Backbone, self).__init__()

    self.blocks = nn.Sequential(
        Block(3, 16),
        # Block(num_inputs[i], num_outputs[i]),
        # Block(num_inputs[i], num_outputs[i])
    )

  def forward(self, x):
    out = self.blocks(x)
    return out

class Classifier(nn.Module):
  def __init__(self, num_inputs, num_classes):
    super(Classifier, self).__init__()
    self.num_inputs = num_inputs

    # self.Fltn = nn.Flatten()

    # softmax regression
    self.Linear1 = nn.Linear(num_inputs, num_classes)
    nn.init.normal_(self.Linear1.weight, std=0.01)
    nn.init.zeros_(self.Linear1.bias)

  def forward(self, x):
    # x = self.Fltn(x)

    x = x.view(-1, 3*32*32)
    x = self.Linear1(x)
    return x

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.backbone = Backbone()
    self.classifier = Classifier(3*32*32, 10)

  def forward(self, x):
    x = self.backbone(x)
    x = self.classifier(x)
    return x



In [20]:
model = Net()

# Task 3 - Create the Loss and Optimizer

In [21]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)

# Task 4 - Write the training script

In [24]:
# Hyperparameters
num_epochs = 2
batch_size = 32

# Training loop
for epoch in range(num_epochs):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    # Get inputs and labels
    inputs, labels = data

    model.train()

    # Zero gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(inputs)
    l = loss(outputs, labels)
    l.backward()

    # Update optimizer weights
    optimizer.step()

    # Print statistics
    running_loss += l.item()
    if i % 100 == 0:  # Print every 100 mini-batches
      print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i, running_loss / 100))
    running_loss = 0.0

[1,     0] loss: 0.025
[1,   100] loss: 0.022
[1,   200] loss: 0.023
[1,   300] loss: 0.023
[1,   400] loss: 0.032
[1,   500] loss: 0.698
[1,   600] loss: 0.067
[1,   700] loss: 0.022
[1,   800] loss: 0.024
[1,   900] loss: 0.021
[1,  1000] loss: 0.023
[1,  1100] loss: 0.024
[1,  1200] loss: 0.022
[1,  1300] loss: 0.022
[1,  1400] loss: 0.031
[1,  1500] loss: 0.035
[2,     0] loss: 0.030
[2,   100] loss: 0.034
[2,   200] loss: 0.037
[2,   300] loss: 0.028
[2,   400] loss: 0.658
[2,   500] loss: 0.036
[2,   600] loss: 0.028
[2,   700] loss: 0.023
[2,   800] loss: 0.024
[2,   900] loss: 0.023
[2,  1000] loss: 0.016
[2,  1100] loss: 0.022
[2,  1200] loss: 0.026
[2,  1300] loss: 0.021
[2,  1400] loss: 0.029
[2,  1500] loss: 0.031


# Task 5 - Final Model Accuracy on the Validation Set

In [25]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


Accuracy of the network on the 10000 test images: 30 %
